# MLP com PyTorch (roteiro + código)
Introdução ao PyTorch para construção de redes: compreender a abstração tensor + autograd e treinar uma MLP simples, salvando e carregando o modelo.

> Este notebook foi escrito para ser **didático** e **autocontido**. Rode célula a célula durante a aula ou use como material de estudo.


In [ ]:
# Verificação rápida
try:
    import torch, torch.nn as nn, torch.optim as optim
    TORCH_OK = True
    print("PyTorch:", torch.__version__)
except Exception as e:
    TORCH_OK = False
    print("PyTorch não está disponível neste ambiente.\n"
          "O notebook permanece útil para estudo; execute localmente onde o PyTorch esteja instalado.")



## 1) Conceito central do PyTorch: **tensor + autograd**

- **Tensor** é a estrutura de dados principal (como `ndarray` do NumPy), com suporte a GPU.
- **Autograd** registra operações e calcula **gradientes automaticamente** via `.backward()`.


In [ ]:
import numpy as np
if TORCH_OK:
    import torch

# Comparação rápida NumPy x Torch
a = np.array([1.0, 2.0, 3.0])
print("NumPy * 2:", a * 2)

if TORCH_OK:
    b = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
    print("Torch * 2:", b * 2)

    # Exemplo de autograd
    x = torch.tensor(2.0, requires_grad=True)
    y = x ** 2           # y = x^2
    y.backward()         # dy/dx = 2x -> 4
    print("x.grad =", x.grad.item())
else:
    print("Demonstração do autograd não executada (PyTorch indisponível).")



## 2) Construindo modelos com `nn.Module`

A ideia é compor blocos reutilizáveis: camadas (`nn.Linear`), ativações (`nn.ReLU`, `nn.Sigmoid`) e um `forward` que descreve o fluxo.


In [ ]:
if TORCH_OK:
    import torch.nn as nn

    class Perceptron(nn.Module):
        def __init__(self, in_features):
            super().__init__()
            self.fc = nn.Linear(in_features, 1)
            self.act = nn.Sigmoid()
        def forward(self, x):
            return self.act(self.fc(x))

    class MLP_Iris(nn.Module):
        def __init__(self, in_features=4, hidden=8, out_features=3):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(in_features, hidden),
                nn.ReLU(),
                nn.Linear(hidden, out_features)  # logits
            )
        def forward(self, x):
            return self.net(x)  # CrossEntropyLoss espera logits (sem Softmax)

    print("Exemplos de modelos prontos:")
    print(Perceptron(4))
    print(MLP_Iris())
else:
    print("Definições de modelo não executadas (PyTorch indisponível).")



## 3) Dataset simples: **Iris** (scikit-learn)

Usaremos o Iris para classificar 3 espécies. É um dataset pequeno e clássico — perfeito para uma primeira MLP.


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target.astype(np.int64)  # 0,1,2

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train).astype(np.float32)
X_test = scaler.transform(X_test).astype(np.float32)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:
if TORCH_OK:
    import torch
    from torch.utils.data import TensorDataset, DataLoader

    X_train_t = torch.from_numpy(X_train)
    y_train_t = torch.from_numpy(y_train)
    X_test_t  = torch.from_numpy(X_test)
    y_test_t  = torch.from_numpy(y_test)

    train_ds = TensorDataset(X_train_t, y_train_t)
    test_ds  = TensorDataset(X_test_t, y_test_t)

    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
    test_loader  = DataLoader(test_ds, batch_size=32)
    print("Loaders prontos.")
else:
    print("Dataloaders não criados (PyTorch indisponível).")



## 4) Loop de treinamento (SGD + CrossEntropyLoss)

Fluxo: **forward → loss → backward → step**.  
A CrossEntropyLoss já inclui o Softmax internamente (por isso usamos logits na saída do modelo).


In [ ]:
import math, time

if TORCH_OK:
    model = MLP_Iris(in_features=4, hidden=8, out_features=3)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.05)

    def accuracy(loader):
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for xb, yb in loader:
                logits = model(xb)
                preds = logits.argmax(dim=1)
                correct += (preds == yb).sum().item()
                total += yb.size(0)
        return correct / total

    EPOCHS = 120
    history = {"loss": [], "acc_train": [], "acc_test": []}

    t0 = time.time()
    for epoch in range(1, EPOCHS+1):
        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        tr_acc = accuracy(train_loader)
        te_acc = accuracy(test_loader)
        history["loss"].append(epoch_loss)
        history["acc_train"].append(tr_acc)
        history["acc_test"].append(te_acc)

        if epoch % 20 == 0 or epoch == 1:
            print(f"Época {epoch:3d}/{EPOCHS} | loss={epoch_loss:.4f} | acc_tr={tr_acc:.3f} | acc_te={te_acc:.3f}")
    t1 = time.time()
    print(f"Treino concluído em {t1 - t0:.2f}s")
else:
    print("Treinamento não executado (PyTorch indisponível).")


In [ ]:
import matplotlib.pyplot as plt

if TORCH_OK and len(history["loss"]) > 0:
    plt.figure(figsize=(10,4))
    plt.plot(history["loss"], label="Loss (train)")
    plt.xlabel("Época"); plt.ylabel("Loss"); plt.title("Curva de perda"); plt.grid(True); plt.legend()
    plt.show()

    plt.figure(figsize=(10,4))
    plt.plot(history["acc_train"], label="Acurácia treino")
    plt.plot(history["acc_test"], label="Acurácia teste")
    plt.xlabel("Época"); plt.ylabel("Acurácia"); plt.title("Evolução da acurácia"); plt.grid(True); plt.legend()
    plt.show()
else:
    print("Sem histórico para plotes (PyTorch indisponível).")



## 5) Salvar e carregar o modelo

Use `state_dict` para salvar apenas os **pesos** (boa prática para compartilhar).


In [ ]:
import os

if TORCH_OK:
    save_path = "mlp_iris_state_dict.pth"
    torch.save(model.state_dict(), save_path)
    print("Modelo salvo em:", save_path)

    # Carregar em um novo objeto com a mesma arquitetura
    reloaded = MLP_Iris(in_features=4, hidden=8, out_features=3)
    reloaded.load_state_dict(torch.load(save_path, map_location="cpu"))
    reloaded.eval()
    print("Modelo recarregado. Exemplo de print(model):")
    print(reloaded)
else:
    print("Salvar/carregar não executado (PyTorch indisponível).")



## 6) Visualização da rede

- `print(model)` exibe a **estrutura das camadas**.
- Para visualizar o **grafo computacional**, é possível usar a biblioteca externa `torchviz`.

> Exemplo (opcional, requer `torchviz` instalado):
```python
from torchviz import make_dot
x = torch.randn(1, 4)
make_dot(model(x), params=dict(model.named_parameters()))
```



## 7) Conclusão orientada ao negócio

Mesmo com uma MLP pequena, observamos a redução do erro e bom desempenho no conjunto de teste. Na prática, modelos assim podem apoiar tarefas como **classificação de perfis** e **priorização de ações**. Ao salvar e recarregar o modelo, abrimos caminho para **implantação** e **reprodutibilidade**, permitindo que times usem o modelo em outros sistemas e validem resultados de forma consistente.
